In [1]:
import pandas as pd
import json
file = open("./resources/basket.csv", "r")

In [2]:
data = [0] * 200
i = 0
for line in file:
    l = line.rstrip('\n')
    l = l.split(',')
    if l[0] == '': continue
    data[i] = l
    i += 1

length 1 itemsets

In [3]:
minSup = 0.3
itemset = {}
for business in data:
    for x in business:
        if x in itemset:
            itemset[x] += 1
        else:
            itemset.update({x: 0})

In [4]:
temp_itemset = {}
for item in itemset:
    if itemset[item] / 200 > minSup:
        temp_itemset.update({item: itemset[item]})
itemset = temp_itemset

length 2 itemsets

In [5]:
temp_itemset = {}
used = []
for item1 in itemset:
    used.append(item1)
    for item2 in itemset:
        if item2 in used: 
            continue
        else: 
            merge = str(item1 + ',' + item2)
            temp_list = merge.split(',')
            for business in data:
                if all(a in business for a in temp_list):
                    if merge in temp_itemset:
                        temp_itemset[merge] += 1
                    else:
                        temp_itemset.update({merge: 0})
itemset = temp_itemset

In [6]:
temp_itemset = {}
for item in itemset:
    if itemset[item] / 200 > minSup:
        temp_itemset.update({item: itemset[item]})
itemset = temp_itemset

length 3 itemsets

In [7]:
def attempt_merge(a, b):
    temp1 = a.split(',')
    temp2 = b.split(',')
    for i in range(len(temp1)-1):
        if temp1[i] != temp2[i]:
            return 0
    ret = ''
    for i in temp1:
        ret += i + ','
    ret += temp2[len(temp2)-1]
    return ret

In [8]:
def gen_candidate(itemset):
    temp_itemset = {}
    used = []
    for item1 in itemset:
        used.append(item1)
        for item2 in itemset:
            if item2 in used: 
                continue
            else: 
                merge = attempt_merge(item1, item2)
                if merge == 0:
                    continue
                temp_list = merge.split(',')
                for business in data:
                    if all(a in business for a in temp_list):
                        if merge in temp_itemset:
                            temp_itemset[merge] += 1
                        else:
                            temp_itemset.update({merge: 0})
    ret_itemset = {}
    for item in temp_itemset:
        if temp_itemset[item] / 200 > minSup:
            ret_itemset.update({item: temp_itemset[item]})
    return ret_itemset

In [9]:
for i in range(3):
    old_itemset = itemset.copy()
    itemset = gen_candidate(itemset)

In [10]:
confidence = {}
for item in itemset:
    for old_item in old_itemset:
        if old_item in item:
            confidence.update({item: itemset[item]/old_itemset[old_item]})

In [11]:
itemset

{'is_open,RestaurantsTakeOut,RestaurantsDelivery,Ambience,BusinessAcceptsCreditCards': 106,
 'is_open,RestaurantsTakeOut,RestaurantsDelivery,Ambience,OutdoorSeating': 105,
 'is_open,RestaurantsTakeOut,RestaurantsDelivery,Ambience,BusinessParking': 102,
 'is_open,RestaurantsTakeOut,RestaurantsDelivery,Ambience,RestaurantsPriceRange2': 104,
 'is_open,RestaurantsTakeOut,RestaurantsDelivery,Ambience,RestaurantsReservations': 106,
 'is_open,RestaurantsTakeOut,RestaurantsDelivery,Ambience,RestaurantsGoodForGroups': 104,
 'is_open,RestaurantsTakeOut,RestaurantsDelivery,Ambience,HasTV': 102,
 'is_open,RestaurantsTakeOut,RestaurantsDelivery,BusinessAcceptsCreditCards,OutdoorSeating': 110,
 'is_open,RestaurantsTakeOut,RestaurantsDelivery,BusinessAcceptsCreditCards,BusinessParking': 105,
 'is_open,RestaurantsTakeOut,RestaurantsDelivery,BusinessAcceptsCreditCards,RestaurantsPriceRange2': 109,
 'is_open,RestaurantsTakeOut,RestaurantsDelivery,BusinessAcceptsCreditCards,RestaurantsReservations': 107,

In [12]:
confidence

{'is_open,RestaurantsTakeOut,RestaurantsDelivery,Ambience,BusinessAcceptsCreditCards': 0.7464788732394366,
 'is_open,RestaurantsTakeOut,RestaurantsDelivery,Ambience,OutdoorSeating': 0.7241379310344828,
 'is_open,RestaurantsTakeOut,RestaurantsDelivery,Ambience,BusinessParking': 0.7132867132867133,
 'is_open,RestaurantsTakeOut,RestaurantsDelivery,Ambience,RestaurantsPriceRange2': 0.7323943661971831,
 'is_open,RestaurantsTakeOut,RestaurantsDelivery,Ambience,RestaurantsReservations': 0.7310344827586207,
 'is_open,RestaurantsTakeOut,RestaurantsDelivery,Ambience,RestaurantsGoodForGroups': 0.7323943661971831,
 'is_open,RestaurantsTakeOut,RestaurantsDelivery,Ambience,HasTV': 0.7132867132867133,
 'is_open,RestaurantsTakeOut,RestaurantsDelivery,BusinessAcceptsCreditCards,OutdoorSeating': 0.738255033557047,
 'is_open,RestaurantsTakeOut,RestaurantsDelivery,BusinessAcceptsCreditCards,BusinessParking': 0.7291666666666666,
 'is_open,RestaurantsTakeOut,RestaurantsDelivery,BusinessAcceptsCreditCards,Re

In [28]:
temp = []
for c in confidence:
    temp.append(confidence[c])
temp.sort()
temp = temp[-5:] + temp[:5]

In [33]:
for t in temp:
    for c in confidence: 
        if confidence[c] == t:
            temp_set = c.split(',')
            print('Rule:', str(temp_set[:len(temp_set)-2]), '->', temp_set[len(temp_set)-1] , '| confidence =', t)
            break       

Rule: ['RestaurantsTakeOut', 'RestaurantsDelivery', 'OutdoorSeating'] -> stars > 3.5 | confidence = 1.0
Rule: ['RestaurantsTakeOut', 'RestaurantsDelivery', 'OutdoorSeating'] -> stars > 3.5 | confidence = 1.0
Rule: ['RestaurantsTakeOut', 'RestaurantsDelivery', 'OutdoorSeating'] -> stars > 3.5 | confidence = 1.0
Rule: ['RestaurantsTakeOut', 'RestaurantsDelivery', 'OutdoorSeating'] -> stars > 3.5 | confidence = 1.0
Rule: ['RestaurantsTakeOut', 'RestaurantsDelivery', 'OutdoorSeating'] -> stars > 3.5 | confidence = 1.0
Rule: ['is_open', 'OutdoorSeating', 'BusinessParking'] -> HasTV | confidence = 0.6985294117647058
Rule: ['is_open', 'RestaurantsTakeOut', 'OutdoorSeating'] -> RestaurantsGoodForGroups | confidence = 0.6993006993006993
Rule: ['is_open', 'RestaurantsTakeOut', 'BusinessParking'] -> RestaurantsGoodForGroups | confidence = 0.7
Rule: ['is_open', 'RestaurantsTakeOut', 'BusinessParking'] -> RestaurantsGoodForGroups | confidence = 0.7
Rule: ['is_open', 'RestaurantsTakeOut', 'Restauran